In [1]:
import functions as mf
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as la
from scipy.special import lambertw

import importlib
importlib.reload(mf)

<module 'functions' from 'c:\\Users\\brent\\Documents\\comp_multphase_flow\\functions.py'>

In [2]:
rho_plasma = 1003 #  kg/m3
mu_plasma = 1e-3 # kg/ms

rho_RBC = 1096 # kg/m3
diameter_RBC = 8e-6 # m

y_end = 5e-3 # m
Ny = 100

boundary_condition = [1, 1] 
blood_flow_rate = 20e-3 # m/s  # moet gecheckt worden
pressure_difference = 1
pressure_boundary = [1, 1]
system = mf.Multiflow(Ny, y_end, rho_plasma, pressure_difference,
                      pressure_boundary, boundary_condition, mu_plasma)
mu = system.calc_mu(mu_plasma)

# Plan van aanpak
1D model
1. define system parameters
2. solve laminar
3. solve prantdl in while loop
4. solve wall function in while loop
5. solve alpha2
6. solve velocity u

In [4]:
# Prescribed flow rate
Q_des = np.pi * (system.y_end/2)**2 * blood_flow_rate
eps = 1e-3
error = 1
system.set_pressure_difference(pressure_difference)

velocity = system.simulate(mu)
Q = np.trapz(velocity, x=system.y)
factor = 0.5
#P = [system.pressure_difference, 2*system.pressure_difference]
switch = [0,0]

while error > eps:
    if switch[-2] != switch[-1]:
        factor*=0.8
    if Q<Q_des:
        # system.pressure_difference = P[-1] + np.abs(P[-1]-P[-2])/2 * factor
        system.pressure_difference -= system.pressure_difference * factor

        velocity = system.simulate(mu)
        Q = np.trapz(velocity, x=system.y)
        error = np.abs(Q - Q_des)
        switch.append(0)
        # P.append(system.pressure_difference)
    else:
        # system.pressure_difference = P[-1] - np.abs(P[-1]-P[-2])/2 * fac
        system.pressure_difference += system.pressure_difference * factor
        velocity = system.simulate(mu)
        Q = np.trapz(velocity, x=system.y)
        error = np.abs(Q-Q_des)
        # P.append(system.pressure_difference)
        switch.append(1)
    if len(switch) == 10:
        print("Oh nyo, it's bwoken")
        break

plt.plot(velocity, system.y)
print(Q, Q_des)

IndexError: tuple index out of range